### Name :  W16 作業2
### Function : Application of NER and Topic Model 

#### input data : 賴士葆.csv

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import gc
import math
import jieba
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter

##  function Extract_NER():   Extract NER from receiving text list

In [2]:
def Extract_NER(ls_textRow):
    from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
    
    # 載入 自訂詞典
    User_Dict = {}
    with open("KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
        for udic in UDicts:
            udWord = udic.strip().split(" ")
            if len(udWord) > 1:
                User_Dict[udWord[0]] = udWord[1]
            else:
                User_Dict[udWord[0]] = 10    # 未給定權重者一律賦予預設值 10                
    dictionary = construct_dictionary(User_Dict)
 
    ws = WS("./data")
    pos = POS("./data")
    ner = NER("./data")
    
    #  ls_segRow = ws(ls_textRow) ------------   without User Dictionary
    ls_segRow = ws(ls_textRow, 
                   sentence_segmentation=True,
                   segment_delimiter_set = {",", "。", ":", "?", "!", ";", "、"},
                   coerce_dictionary = dictionary)
    #print("ls_segRow = ", ls_segRow)
    #print("-----------------------------")
    
    del ws
    del WS
    gc.collect()
    ls_posRow = pos(ls_segRow)
    del pos
    del POS
    gc.collect()
    ls_nerRow = ner(ls_segRow,ls_posRow)
    del ner
    del NER
    gc.collect()
    #print("ls_nerRow = ", ls_nerRow)
    #print("-----------------------------")
    return ls_nerRow

## 1. Read data from csv or xlsx file

In [3]:
df = pd.read_csv('賴士葆.csv',encoding='utf-8') #匯資料
df.head(1)

,Unnamed: 0,屆期,會期,會議日期,委員姓名,逐字稿
0,37,8,6,9/16/2014,賴士葆,﻿賴委員士葆：（14 時 1 分）主席、各位同仁。本席等 19 人，查隨著智慧型電器設備不斷...


## 2.  Main function:
### 2.1  Pass text data to function "Extract_NER" by row
### 2.2  Integrate every 10 NER return rows into one data frame  
### 2.3  Write frame into small csv after droping duplicate, locating class, filtering short NER, sorting by class.. 

In [4]:
#逐筆傳遞欄位 "text"
lai_text = df["逐字稿"]
print("逐字稿 欄位筆數 = ",len(lai_text))
print("------------------------------------------")
ls_class = list()
ls_ner = list()
fileCunt = 0
rowCunt = 0
for i in range(len(lai_text)):
    rowCunt += 1
    ls_dfRow = [lai_text[i]]
    # print("ls_dfRow = ", ls_dfRow)
    print("Row No = ",str(i))
    entity_list = Extract_NER(ls_dfRow)
    for j in range(len(entity_list[0])):
        seg = entity_list[0].pop()
        # print("seg = ",seg)
        ls_class.append(seg[2])
        ls_ner.append(seg[3])
    #print("===========================================")
    #print("ls_class = ",ls_class)
    #print("ls_ner = ",ls_ner)
    # if i == 5 or rowCunt == 2:
    if i == (len(lai_text)-1) or rowCunt == 10:
        fileCunt += 1
        extract_df = pd.DataFrame({'Class':ls_class, 'NER':ls_ner})
        deDup_df = extract_df.drop_duplicates().copy()
        del extract_df #去除重複
        cate_df = deDup_df.loc[deDup_df['Class'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE','NORP'])].copy()
        del deDup_df
        filt_df = cate_df[cate_df['NER'].map(len) >= 2].copy()
        del cate_df
        NER_df = filt_df.sort_values(by=['Class']).copy()
        del filt_df
        #print("__________________________________________________________")  
        #print(NER_df)
        print("fileCunt No = ",str(fileCunt))
        NER_df.to_csv(r"Lai Ner out/Lai_" + str(fileCunt)+ ".csv")
        #NER_df.to_csv(r"ner out/NerTest_" + str(fileCunt)+ ".csv")
        rowCunt = 0
print("=====================================================================")
print("Totle NER Files = ",fileCunt)


逐字稿 欄位筆數 =  622
------------------------------------------
Row No =  0


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  1
Row No =  2
Row No =  3
Row No =  4
Row No =  5
Row No =  6
Row No =  7
Row No =  8
Row No =  9
fileCunt No =  1
Row No =  10


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  11
Row No =  12
Row No =  13
Row No =  14
Row No =  15
Row No =  16
Row No =  17
Row No =  18
Row No =  19
fileCunt No =  2
Row No =  20


...

Row No =  601
Row No =  602
Row No =  603
Row No =  604
Row No =  605
Row No =  606
Row No =  607
Row No =  608
Row No =  609
fileCunt No =  61
Row No =  610


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  611
Row No =  612
Row No =  613
Row No =  614
Row No =  615
Row No =  616
Row No =  617
Row No =  618
Row No =  619
fileCunt No =  62
Row No =  620


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  621
fileCunt No =  63
Totle NER Files =  63


## 3. Integrate all csv files into single data frame

In [6]:
import glob
# import pandas as pd

path = r'Lai Ner out'
#path = r'ner out'
allFiles = glob.glob("{}/*.csv".format(path))
li = []
for filename in allFiles:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

nerFrame = pd.concat(li, axis=0, ignore_index=True)
print(len(nerFrame))
nerFrame.groupby('Class').count()

80959


,NER
Class,
EVENT,2043
GPE,10155
LAW,2471
LOC,2379
NORP,3232
ORG,33810
PERSON,26869


## 4. Drop duplicates, group by class and write into csv file 

In [7]:
ddFrame = nerFrame.drop_duplicates().copy()
print(len(ddFrame))
ddFrame.groupby('Class').count()

2288


,NER
Class,
EVENT,60
GPE,271
LAW,76
LOC,64
NORP,85
ORG,944
PERSON,788


In [8]:
ddFrame.sort_values(by=['Class']).to_csv(r"Lai Ner out/Ligis_NER.csv")
#ddFrame.sort_values(by=['Class']).to_csv(r"ner out/NerTest.csv")

In [9]:
df = pd.read_csv('賴士葆.csv',encoding='utf-8') #匯資料
df.head(1)

,Unnamed: 0,屆期,會期,會議日期,委員姓名,逐字稿
0,37,8,6,9/16/2014,賴士葆,﻿賴委員士葆：（14 時 1 分）主席、各位同仁。本席等 19 人，查隨著智慧型電器設備不斷...


#### 用賴士葆的資料跑TOPIC MODEL

In [10]:
text_list = list(df['逐字稿']) # 把所有message轉成一個list 用for iterate

lai_str = ','.join(text_list)
print(lai_str) #轉字串斷詞

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import gc
import tensorflow as tf
# import keras

# 載入 ckiptagger 自訂詞典
User_Dict = {}
with open("KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
    for udic in UDicts:
        udWord = udic.strip().split(" ")
        if len(udWord) > 1:
            User_Dict[udWord[0]] = udWord[1]
        else:
            User_Dict[udWord[0]] = 100    # 未給定權重者一律賦予預設值 100               
dictionary = construct_dictionary(User_Dict)

In [12]:
# 載入模型
ws = WS("./data")
# pos = POS("./data")
# ner = NER("./data")

C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [13]:
# 載入 StopWord
stopword_list = []
with open("KCC Data/Dict/KccStopWord2020.txt","r", encoding='utf-8-sig') as stopwords:
    for stopword in stopwords:
        stopword_list.append(stopword.strip())

In [14]:
# 開始斷詞
# sentence_segmentation = True,  => To consider delimiters
# segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
# recommend_dictionary = dictionary1, # words in this dictionary are encouraged
# coerce_dictionary = dictionary2, # words in this dictionary are forced
strCut = ws([lai_str], sentence_segmentation=True,
                 segment_delimiter_set = {'"',"\r\n","，","「","」",",","？",'\n', "。", ":", "?", "!", ";", "、"},
                 coerce_dictionary = dictionary)

print("------------CKIPTagger Word Segment ws(lai_str,....) + 刪除標點及停用字 ---------------")
stayed_lst= [] #斷好字的
for word in strCut[0]:
     # 刪除停用字
    if word not in stopword_list:
        stayed_lst.append(word)
        
print(" ".join(stayed_lst))

------------CKIPTagger Word Segment ws(lai_str,....) + 刪除標點及停用字 ---------------
﻿ 賴 委員 士葆 14   時  1  分 位 同仁 本席  19   查隨 智慧型 電器 設備 推陳出新 產品 普及 民眾 手  3C  設備 越來越 智慧型 手機 數位 相機 
 面對 各式各樣  3C  設備 充電 接頭 便利 容易 
 充電 設備 氾濫 影響 環境 生態 歐洲 議會 民國  103   年 法案 
 智慧型 電器 充電器 插頭 規格 統一 預計  3  年 生效 提高 民眾  3C
 產品 便利性 世界 趨勢 接軌 爰 提案 行政院 相關 單位 限期 月 智慧
 型 電器 充電 插頭 規格 統一 提出 相關 辦法 有當 敬請 公決 ﻿ 請 賴 委員 士葆 發言 
 
 賴 委員 士葆 列席 官員 位 同仁 張 部長 提到 最近 股市 量縮 阿里巴巴 關係 
 
 阿里巴巴 吸掉 錢 請問 主委  
 
 請 金管會 主任委員  
 主任委員 銘宗 委員 阿里巴巴 美國 掛牌 吸收 美國 資金  
 賴 委員 士葆 吸 上兆 喔  
 主任委員 銘宗 國內 投資 分到  
 賴 委員 士葆 張 部長 亂 栽贓  
 請 財政部 張 部長  
 張 部長 盛和 委員 栽贓 說 分析師  
 
 賴 委員 士葆 張盛和 講 分析師 講 請問 主委 台灣 有沒有 阿里巴巴
 
  
 
 主任委員 銘宗 定義 阿里巴巴 發展 中國 生態 環境 中國 大
 
 陸 金融 發展 先進 國家 台灣 給予 阿里巴巴 生存空間  
 
 賴 委員 士葆 主產品  C2C  淘寶網 網路 商城 天貓 第三 支付 支付 寶
 
 餘額寶 餘額寶 用戶 高達 八千多萬 阿里巴巴 金融 塊 創新  
 
 主任委員 銘宗 錯  
 賴 委員 士葆 說 塊 台灣 餘額寶 東西 台灣 做 請問  
 主任委員 銘宗 餘額寶 支付 寶 項 產品 支付 支付 寶 三方 支付 功能  
 賴 委員 士葆 剩 錢 利率 高 一點 年 收益 達  10%  
 主任委員 銘宗 委員 報告 支付 寶 三方 支付 賣 金融 產品 中國 大陸  
 賴 委員 士葆 不可以 有沒有 放  
 
 主任委員 銘宗 特別 業者 想 開

In [15]:
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel

In [16]:
fList = []
with open("賴士葆_output.txt","r", encoding='utf-8-sig') as files:
     for file in files:
        #simple_preprocess: tokenize,去除標點 - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15) 
        fList.append(list(utils.simple_preprocess(file, deacc=True, min_len=2,)))
print("fList[:2] = ", fList[:2])
print("------------------------------")

fList[:2] =  [['jieba_results'], ['委員', '士葆', '主席', '各位', '同仁', '本席', '隨著', '智慧型', '電器設備', '不斷', '推陳出新', '產品', '日漸', '普及', '民眾', '手邊', '設備', '越來越', '例如', '智慧型', '手機', '數位', '相機', '面對', '各式各樣', '設備', '往往', '使用', '不同', '充電', '接頭', '不僅', '造成', '使用', '便利', '容易', '造成', '充電', '設備', '影響', '環境', '生態', '為此', '歐洲議會', '今年', '民國', '通過', '法案', '規定', '所有', '智慧型', '電器', '充電器', '插頭', '規格', '必須', '統一', '預計', '將在', '年內', '生效', '因此', '提高', '民眾', '使用', '產品', '便利性', '並與', '世界', '趨勢', '接軌', '提案', '要求', '行政院', '及其', '相關', '單位', '限期', '三個', '針對', '智慧', '電器', '充電', '插頭', '規格', '統一', '提出', '相關', '辦法', '是否', '有當', '敬請', '公決']]
------------------------------


In [17]:
dict = corpora.Dictionary(fList)
print("dict = ", dict)
print("------------------------------")

dict =  Dictionary(28951 unique tokens: ['jieba_results', '三個', '不僅', '不同', '不斷']...)
------------------------------


In [18]:
# creat mapping matrix of [word_id, word_frequency] for each word segment
corpus = [dict.doc2bow(text) for text in fList]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:1]]
print("======================================================")

corpus[:1] =  [[(0, 1)]]
------------------------------
print lists of [word:frequency]
('jieba_results', 1)


In [19]:
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=10, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("KCC Data/NewsCisTest-KccDict2020-LDA-TopicNum10_lai.txt", 'w', encoding='utf-8-sig')
lstTM = lda_model.print_topics(num_topics=10, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(fList), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)

strTM =  00.075*"院長" + 0.027*"貞昌" + 0.012*"民進黨" + 0.011*"總統" + 0.008*"人民" + 0.008*"英文" + 0.008*"清德" + 0.006*"各位" + 0.006*"為什麼" + 0.006*"清楚" + 0.005*"主委" + 0.005*"這是" + 0.005*"志嘉" + 0.005*"一個" + 0.005*"中華民國" + 0.005*"政治" + 0.004*"民主" + 0.004*"退輔會" + 0.004*"德發" + 0.004*"教育部"

strTM =  10.052*"委員" + 0.041*"士葆" + 0.027*"我們" + 0.020*"部長" + 0.018*"沒有" + 0.013*"這個" + 0.012*"現在" + 0.011*"就是" + 0.011*"可以" + 0.011*"你們" + 0.010*"因為" + 0.009*"所以" + 0.009*"這樣" + 0.008*"不是" + 0.008*"主席" + 0.007*"不要" + 0.007*"知道" + 0.007*"一個" + 0.006*"什麼" + 0.006*"次長"

strTM =  20.011*"提案" + 0.011*"資安" + 0.009*"相關" + 0.008*"是否" + 0.008*"要求" + 0.007*"單位" + 0.007*"同仁" + 0.006*"行政院" + 0.006*"各位" + 0.006*"比照" + 0.005*"line" + 0.005*"手機" + 0.005*"安全" + 0.005*"主席" + 0.005*"保障" + 0.005*"有當" + 0.005*"敬請" + 0.004*"針對" + 0.004*"open" + 0.004*"公決"

strTM =  30.000*"私產" + 0.000*"電和鴻海差" + 0.000*"降營" + 0.000*"名符其實" + 0.000*"名符" + 0.000*"恐日" + 0.000*"結語" + 0.000*"經營不善" + 0.000*"西德" + 0.000*"鎮長" + 0.000*"陽謀" + 0.000*"預言" + 0.000*"黨史

In [20]:
import pyLDAvis
import pyLDAvis.gensim #import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

In [21]:
pyLDAvis.enable_notebook
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict, doc_topic_dist=None)
"""
参数说明：
prepare(topic_model, corpus, dictionary, doc_topic_dist=None)
topic_model：训练得到的 gensim LdaModel 对象，不支持其他 gensim 模型类型
corpus：以词袋形式表示的语料（用于训练主题模型的语料）
dictionary：用于构建词袋模型的字典，即 gensim Dictionary 对象
doc_topic_dist：可选参数，用于传入 LDA 模型的文档主题分布，默认取值为 None，当需要多次调用 prepare 函数时，可以传入该参数
"""
pyLDAvis.display(vis)

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
